In [1]:
#import libaries
import requests
from bs4 import BeautifulSoup
from urllib.request import urlretrieve
import numpy as np
import pandas as pd

In [2]:
#import libaries
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
import re

In [7]:
#used selenium and chrome to automate a web browser to scroll through webpage since it couldn't be scraped by normal means

#append links to each product on the website to list 'Wow'
wow = []

driver = webdriver.Chrome('C:\\Users\\akomandu\\Python_Stuff\\chromedriver.exe')
driver.get("https://living-future.org/declare/")
driver.maximize_window()

SCROLL_PAUSE_TIME = 1.0

# Get scroll height
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    page = driver.page_source
    soup = BeautifulSoup(page, 'html.parser')
    wow.append(soup.find_all('a', href=True))
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)

    # Calculate new scroll height and compare with last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height

In [8]:
Links = []

#filter out extraneous links
for i in wow[-1]:
    if 'lf-reveal-card' in str(i):
        Links.append(i['href'])

In [10]:
Component = []
Ingredient = []
CAS = []
Percent = []
Source = []
Link = []
Declare_ID = []
License_Expiration = []
Declaration_Status = []
Final_Assembly_Location = []
Life_Expectancy = []
EOL = []
VOC = []
CDPH = []
Product = []
Company = []
File = []

for i in Links:
    print(i)
    page = requests.get(i)
    time.sleep(5)
    soup = BeautifulSoup(page.content, 'html.parser')
    table=soup.find('table')

    company = soup.find('strong').get_text()
    link = i
    extra = []
    for i in soup.findAll("div", {"class": "declare-option"}):
        extra.append(i.get_text().split('\n'))

    table_rows = table.find_all('tr')
    for tr in table_rows:

        td = tr.find_all('td')
        row = [re.sub(' +',' ',i.text) for i in td]
        if len(row) > 0:
            Component.append(row[0])
            Ingredient.append(row[1])
            CAS.append(row[2])
            Percent.append(row[3])
            Source.append(row[4])
            File.append(link.split('/')[-2] + '.pdf')
            Company.append(company)
            Link.append(link)
            Product.append(soup.find('h3').get_text())
            for i in extra:
                if 'Declare ID' in i:
                    Declare_ID.append(i[2:-1])
                if 'License Expiration' in i:
                    License_Expiration.append(i[2:-1])
                if 'Declaration Status' in i:
                    Declaration_Status.append(i[2:-1])
                if 'Final Assembly Location' in i:
                    Final_Assembly_Location.append(i[3:-2])
                if 'Life Expectancy' in i:
                    Life_Expectancy.append(i[2:-1])
                if 'End of Life' in i:
                    EOL.append(i[2:-1])
                if 'VOC Content in g/L' in i:
                    VOC.append(i[2:-1])
                if 'CDPH Standard Method V1.1 Compliant' in i:    
                    CDPH.append(i[2])

https://living-future.org/declare-products/1-1-2-drywall-suspension-system/
https://living-future.org/declare-products/10-mm-gib-aqualine-plasterboard-2/
https://living-future.org/declare-products/1100-cross-piece-grille2100-panelized-linear-fsc-douglas-fir/
https://living-future.org/declare-products/1100-cross-piece-grille2100-panelized-linear-fsc-hemlock/
https://living-future.org/declare-products/13-mm-gib-aqualine-plasterboard-2/
https://living-future.org/declare-products/1600-wall-system-1-curtain-wall-and-system-2-curtain-wall/
https://living-future.org/declare-products/1600ut-wall-system-1-curtain-wall-and-system-2-curtain-wall/
https://living-future.org/declare-products/190350500-standard-entrances/
https://living-future.org/declare-products/2500-ut-unitwall-system/
https://living-future.org/declare-products/8000-series-exit-devices/
https://living-future.org/declare-products/9-16-ez-stab-bolt-slot-system-1-4-reveal-9-16-ez-stab-bolt-slot-system/
https://living-future.org/decla

In [11]:
data = pd.DataFrame({
    'Component': Component,
    'Ingredient' : Ingredient,
    'CAS' : CAS,
    'Percent' : Percent,
    'Source' : Source,

    'Declare_ID' :Declare_ID,
    'License_Expiration' : License_Expiration,
    'Declaration_Status' : Declaration_Status,
    'Final_Assembly_Location' : Final_Assembly_Location, 
    'Life_Expectancy' : Life_Expectancy,
    'EOL' : EOL,
    'VOC' : VOC,
    'CDPH' : CDPH,
    'Link' : Link,
    'Company' : Company,
    'Product' : Product,
    'File' : File
})

def delist(x):
    if type(x) == list:
        if len(x) == 1:
            return(''.join(x))
        if len(x) > 1:
            return(', '.join(x))
    else:
        return(x)

data.Component = data.Component[data.Component.str.strip() != '']
data = data.fillna(method='ffill')
data = data.replace({'\n': ''}, regex=True)
data = data.applymap(lambda x: delist(x))

In [12]:
data.head()

,CAS,CDPH,Company,Component,Declaration_Status,Declare_ID,EOL,File,Final_Assembly_Location,Ingredient,License_Expiration,Life_Expectancy,Link,Percent,Product,Source,VOC
0,65997-19-5,N/A,,1-1/2” Drywall Suspension System,Red List Free,"CTG-0016, CTG-0017",Recyclable,1-1-2-drywall-suspension-system.pdf,"Bradenton, FL, USA, Vaughan, ON, Canada","Welding Fume, Stainless Steel",01 Jul 2019,30 years,https://living-future.org/declare-products/1-1...,100%,1-1/2” Drywall Suspension System,,N/A
1,13397-24-5,Yes,Winstone Wallboards,Gypsum Wall Panel,Red List Free,WWL-2001,Recyclable,10-mm-gib-aqualine-plasterboard-2.pdf,"Auckland, New Zealand",Gypsum,01 Nov 2018,50 years,https://living-future.org/declare-products/10-...,>90,10 mm GIB Aqualine® Plasterboard,"Ceduna, Victoria, Australia",N/A
2,9004-64-6,Yes,Winstone Wallboards,Gypsum Wall Panel,Red List Free,WWL-2001,Recyclable,10-mm-gib-aqualine-plasterboard-2.pdf,"Auckland, New Zealand",Cellulose Fibre - Paper,01 Nov 2018,50 years,https://living-future.org/declare-products/10-...,<6,10 mm GIB Aqualine® Plasterboard,"Melbourne, Victoria, Australia",N/A
3,65997-17-3,Yes,Winstone Wallboards,Gypsum Wall Panel,Red List Free,WWL-2001,Recyclable,10-mm-gib-aqualine-plasterboard-2.pdf,"Auckland, New Zealand",Fibreglass,01 Nov 2018,50 years,https://living-future.org/declare-products/10-...,<2,10 mm GIB Aqualine® Plasterboard,"Etowah, Tennessee, USA",N/A
4,9005-25-8,Yes,Winstone Wallboards,Gypsum Wall Panel,Red List Free,WWL-2001,Recyclable,10-mm-gib-aqualine-plasterboard-2.pdf,"Auckland, New Zealand",Starch,01 Nov 2018,50 years,https://living-future.org/declare-products/10-...,<1,10 mm GIB Aqualine® Plasterboard,"Altona, Victoria, Australia",N/A


In [13]:
data.to_csv('Living_Future.csv')

In [114]:



Links.index('https://living-future.org/declare-products/paraflex/')

374

In [117]:
len(Product)

6484